In [87]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../numeral_context/')
sys.path.append('../../../')
sys.path.append('../../../utils/')
sys.path.append('../../mag_num/')
sys.path.append('../numeral_prediction/')
import pickle
import numpy as np
import pandas as pd
from number_handler import is_numeral, to_numeral
from tqdm import tqdm_notebook
from create_abstract_embeddings import get_all_target_numerals, random_sample_numerals, create_magnitude_embeds
from numeral_prediction_evaluator import Evaluator
from numeral_to_embed import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
dataset = pickle.load(open('../numeral_prediction/data.pkl','rb'))
res = get_all_target_numerals(dataset)
sampled_res = random_sample_numerals(res)
pickle.dump(sampled_res, open('./magnitudes_numerals.pkl', 'wb'))
magnitudes_numerals = pickle.load(open('./magnitudes_numerals.pkl', 'rb'))

In [89]:
word2idx = pickle.load(open('../exps/title_all_correct/word2idx.pkl', 'rb'))
e1 = Evaluator(
    word2idx,
    dataset[:30000],
)

In [67]:
result_dict = {}

In [68]:
for i in ['2', '3', '5']:
    trained_prototypes = pickle.load(open('../../../data/wikipedia/save/1B30W/prototypes/{}-0005/trained_prototypes_epoch1_{}00_1.0.dat'.format(i, i),'rb'))
    embeds = [load_prototype(i, trained_prototypes) for i in magnitudes_numerals]
    numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim
#     pickle.dump(numeral_embed_i, open('./p{}00_i.emb'.format(i), 'wb'))
#     pickle.dump(numeral_embed_o, open('./p{}00_o.emb'.format(i), 'wb'))
    idx2vec_i = pickle.load(open('../exps/title_all_correct/embed/p-{}00.emb'.format(i), 'rb'))
    idx2vec_o = pickle.load(open('../exps/title_all_correct/embed/p-{}00_o.emb'.format(i), 'rb'))

    e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
    e1.load_idx2vec(idx2vec_i, idx2vec_o)
    e1.compute_norm_factor_all(
        pickle.load(open('../../../data/wikipedia/save/1B30W/prototypes/{}-0005/idx2vec_o_epoch1.dat'.format(i, i),'rb'))
    )
    ranks = e1.evaluate_avg_rank()
    micro_f1, macro_f1 = e1.evaluate_f1()
    result_dict['p-{}00'.format(i)] = [ranks, micro_f1, macro_f1]

In [73]:
for i in ['2', '3', '5']:
    trained_prototypes = pickle.load(open('../../../data/wikipedia/save/1B30W/prototypes_log/{}-0005/trained_prototypes_epoch1_{}00_1.0.dat'.format(i, i),'rb'))
    embeds = [load_prototype(i, trained_prototypes, log_space=True) for i in magnitudes_numerals]
    numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

    idx2vec_i = pickle.load(open('../exps/title_all_correct/embed/p-{}00-log.emb'.format(i), 'rb'))
    idx2vec_o = pickle.load(open('../exps/title_all_correct/embed/p-{}00-log_o.emb'.format(i), 'rb'))

    e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
    e1.load_idx2vec(idx2vec_i, idx2vec_o)
    e1.compute_norm_factor_all(
        pickle.load(open('../../../data/wikipedia/save/1B30W/prototypes_log/{}-0005/idx2vec_o_epoch1.dat'.format(i, i),'rb'))
    )
    ranks = e1.evaluate_avg_rank()
    micro_f1, macro_f1 = e1.evaluate_f1()
    result_dict['p-{}00-log'.format(i)] = [ranks, micro_f1, macro_f1]

In [74]:
for i in [('300','hard'), ('300','soft'),('500','hard'),('500','soft')]:
    trained_prototypes = pickle.load(open('../../../data/wikipedia/save/1B30W/gmms/gmm-{}-rd-{}/trained_gmms_epoch1_{}.dat'.format(i[0], i[1], i[0]),'rb'))
    gmm = pickle.load(open('../../../data/wikipedia/preprocess1B/NumeralAsNumeral30W/gmm/gmm-{}-rd-{}.dat'.format(i[0], i[1]),'rb'))

    embeds = [load_GMM(i, trained_prototypes, gmm) for i in magnitudes_numerals]
    numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

    idx2vec_i = pickle.load(open('../exps/title_all_correct/embed/g-{}-{}.emb'.format(i[0], i[1]), 'rb'))
    idx2vec_o = pickle.load(open('../exps/title_all_correct/embed/g-{}-{}_o.emb'.format(i[0], i[1]), 'rb'))

    e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
    e1.load_idx2vec(idx2vec_i, idx2vec_o)
    e1.compute_norm_factor_all(
        pickle.load(open('../../../data/wikipedia/save/1B30W/gmms/gmm-{}-rd-{}/idx2vec_o_epoch1.dat'.format(i[0], i[1]),'rb'))
    )
    ranks = e1.evaluate_avg_rank()
    micro_f1, macro_f1 = e1.evaluate_f1()
    result_dict['g-{}-{}'.format(i[0], i[1])] = [ranks, micro_f1, macro_f1]

In [75]:
for i in [('300','hard'), ('300','soft'),('500','hard'),('500','soft')]:
    trained_prototypes = pickle.load(open('../../../data/wikipedia/save/1B30W/gmms_log/gmm-{}-rd-{}/trained_gmms_epoch1_{}.dat'.format(i[0], i[1], i[0]),'rb'))
    gmm = pickle.load(open('../../../data/wikipedia/preprocess1B/NumeralAsNumeral30W/gmm_log/gmm-{}-rd-{}.dat'.format(i[0], i[1]),'rb'))

    embeds = [load_GMM(i, trained_prototypes, gmm, log_space=True) for i in magnitudes_numerals]
    numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

    idx2vec_i = pickle.load(open('../exps/title_all_correct/embed/g-{}-{}-log.emb'.format(i[0], i[1]), 'rb'))
    idx2vec_o = pickle.load(open('../exps/title_all_correct/embed/g-{}-{}-log_o.emb'.format(i[0], i[1]), 'rb'))

    e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
    e1.load_idx2vec(idx2vec_i, idx2vec_o)
    e1.compute_norm_factor_all(
        pickle.load(open('../../../data/wikipedia/save/1B30W/gmms_log/gmm-{}-rd-{}/idx2vec_o_epoch1.dat'.format(i[0], i[1]),'rb'))
    )
    ranks = e1.evaluate_avg_rank()
    micro_f1, macro_f1 = e1.evaluate_f1()
    result_dict['g-{}-{}-log'.format(i[0], i[1])] = [ranks, micro_f1, macro_f1]

In [76]:
ivec = pickle.load(open('../../../data/wikipedia/save/1B30W/token/idx2vec_i_epoch1.dat','rb'))
ovec = pickle.load(open('../../../data/wikipedia/save/1B30W/token/idx2vec_o_epoch1.dat','rb'))
w2i = pickle.load(open('../../../data/wikipedia/preprocess1B/NumeralAsTokenUnkNumeral30W/word2idx.dat','rb'))
embeds = [load_TOKEN(i, ivec, ovec, w2i) for i in magnitudes_numerals]
numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

idx2vec_i = pickle.load(open('../exps/title_all_correct/embed/token.emb', 'rb'))
idx2vec_o = pickle.load(open('../exps/title_all_correct/embed/token_o.emb', 'rb'))

e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
e1.load_idx2vec(idx2vec_i, idx2vec_o)
e1.compute_norm_factor_all(
    pickle.load(open('../../../data/wikipedia/save/1B30W/token/idx2vec_o_epoch1.dat','rb'))
)
ranks = e1.evaluate_avg_rank()
micro_f1, macro_f1 = e1.evaluate_f1()
result_dict['token'] =  [ranks, micro_f1, macro_f1]

In [77]:
model_path = '../../../data/wikipedia/save/1B30W/LSTM/sgns_epoch1.pt'
# embeds = [load_LSTM(i, model_path, 300) for i in magnitudes_numerals]
# numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim
# pickle.dump(numeral_embed_i, open('./LSTM_i.emb', 'wb'))
# pickle.dump(numeral_embed_o, open('./LSTM_o.emb', 'wb'))
numeral_embed_i = pickle.load(open('./LSTM_i.emb', 'rb'))
numeral_embed_o = pickle.load(open('./LSTM_o.emb', 'rb'))
idx2vec_i = pickle.load(open('../exps/title_all_correct/embed/LSTM.emb', 'rb'))
idx2vec_o = pickle.load(open('../exps/title_all_correct/embed/LSTM_o.emb', 'rb'))

e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
e1.load_idx2vec(idx2vec_i, idx2vec_o)
e1.compute_norm_factor_all(
    pickle.load(open('../../../data/wikipedia/save/1B30W/LSTM/idx2vec_o_epoch1.dat','rb'))
)
ranks = e1.evaluate_avg_rank()
micro_f1, macro_f1 = e1.evaluate_f1()
result_dict['LSTM'] =  [ranks, micro_f1, macro_f1]

In [96]:
embeds = [load_fixed(i, 300) for i in magnitudes_numerals]
numeral_embed_i, numeral_embed_o = create_magnitude_embeds(embeds) # 8 x embed_dim

idx2vec_i = pickle.load(open('../exps/title_all_correct/embed/Fixed.emb', 'rb'))
idx2vec_o = pickle.load(open('../exps/title_all_correct/embed/Fixed_o.emb', 'rb'))

e1.load_numeral_embeds(numeral_embed_i=numeral_embed_i, numeral_embed_o=numeral_embed_o)
e1.load_idx2vec(idx2vec_i, idx2vec_o)
e1.compute_norm_factor_all(
    pickle.load(open('../../../data/wikipedia/save/1B30W/FIXED/idx2vec_o_epoch1.dat','rb'))
)
ranks = e1.evaluate_avg_rank()
micro_f1, macro_f1 = e1.evaluate_f1()
result_dict['Fixed'] =  [ranks, micro_f1, macro_f1]

In [100]:
e1.show_scores(result_dict, 0).T

,0,1,2
p-200,2.657667,0.334500,0.199259
p-300,2.687867,0.343533,0.206581
p-500,2.661467,0.344900,0.208049
p-200-log,2.663867,0.347433,0.204337
p-300-log,2.598000,0.345367,0.238360
p-500-log,2.647200,0.347567,0.206107
g-300-hard,2.620933,0.352433,0.213434
g-300-soft,2.568900,0.346067,0.244394
g-500-hard,2.539267,0.350233,0.250076
g-500-soft,2.575733,0.350733,0.208335
